# <center>3. 特征工程

<center><img src="https://i.loli.net/2021/10/23/EfkhZ7FT89dCacb.png" alt="image-20211023143401397" style="zoom:50%;" />

&emsp;&emsp;transaction数据集则相对复杂，该数据集是有一张商户数据merchants.csv和两张交易数据表处理后合并得到，该过程如下所示：

<center><img src="https://i.loli.net/2021/10/23/vwXiQcd3sWfqhSY.png" alt="image-20211023144942013" style="zoom:80%;" />

接下来，我们就依据这三张表进行后续操作。

## 一、特征工程

&emsp;&emsp;首先需要对得到的数据进一步进行特征工程处理。一般来说，对于已经清洗完的数据，特征工程部分核心需要考虑的问题就是特征创建（衍生）与特征筛选，也就是先尽可能创建/增加可能对模型结果有正面影响的特征，然后再对这些进行挑选，以保证模型运行稳定性及运行效率。此处使用两种特征衍生的方法，即创建通用组合特征与业务统计特征；并在特征创建完毕后，使用另外一种基础而通用的特征筛选的方法：基于皮尔逊相关系数的Filter方法进行特征筛选。

### 1.通用组合特征创建

#### 1.1 通用组合特征的创建方法

&emsp;&emsp;首先是尝试创建一些通用组合特征。

&emsp;&emsp;所谓通用组合特征，指的是通过统计不同离散特征在不同取值水平下、不同连续特征取值之和创建的特征，并根据card_id进行分组求和。具体创建过程我们可以如下简例来进行理解：

<center><img src="https://i.loli.net/2021/10/23/KUTtb32o9eS1MJW.png" alt="image-20211023153800138" style="zoom:67%;" />

通过该方法创建的数据集，不仅能够尽可能从更多维度表示每个card_id的消费情况，同时也能够顺利和训练集/测试集完成拼接，从而带入模型进行建模。相关过程我们可以借助Python中的字典对象类型来进行实现，上述简例实现过程如下：

In [1]:
import gc
import time
import numpy as np
import pandas as pd
from datetime import datetime

In [4]:
# 借助字典创建DataFrame
d1 = {'card_id':[1, 2, 1, 3], 
      'A':[1, 2, 1, 2], 
      'B':[2, 1, 2, 2], 
      'C':[4, 5, 1, 5], 
      'D':[7, 5, 4, 8],}

t1 = pd.DataFrame(d1)
t1

,card_id,A,B,C,D
0,1,1,2,4,7
1,2,2,1,5,5
2,1,1,2,1,4
3,3,2,2,5,8


In [5]:
# 标注特征类别
numeric_cols = ['C', 'D']
category_cols = ['A', 'B']

In [6]:
# 创建一个以id为key、空字典为value的字典
features = {}
card_all = t1['card_id'].values.tolist()
for card in card_all:
    features[card] = {}

In [7]:
features

{1: {}, 2: {}, 3: {}}

In [8]:
# 所有字段名称组成的list
columns = t1.columns.tolist()
columns

['card_id', 'A', 'B', 'C', 'D']

In [9]:
# 其中card_id在list当中的索引值
idx = columns.index('card_id')
idx

0

In [10]:
# 离散型字段的索引值
category_cols_index = [columns.index(col) for col in category_cols]
category_cols_index

[1, 2]

In [11]:
# 连续型字段的索引值
numeric_cols_index = [columns.index(col) for col in numeric_cols]
numeric_cols_index

[3, 4]

In [12]:
# 对离散型字段的不同取值和连续型字段两两组合
# 同时完成分组求和
for i in range(t1.shape[0]):
    va = t1.loc[i].values
    card = va[idx]
    for cate_ind in category_cols_index:
        for num_ind in numeric_cols_index:
            col_name = '&'.join([columns[cate_ind], str(va[cate_ind]), columns[num_ind]])
            features[card][col_name] = features[card].get(col_name, 0) + va[num_ind]

&emsp;&emsp;然后查看features最终结果

In [13]:
features

{1: {'A&1&C': 5, 'A&1&D': 11, 'B&2&C': 5, 'B&2&D': 11},
 2: {'A&2&C': 5, 'A&2&D': 5, 'B&1&C': 5, 'B&1&D': 5},
 3: {'A&2&C': 5, 'A&2&D': 8, 'B&2&C': 5, 'B&2&D': 8}}

能够发现，此时features就是一个已经包含了离散变量的不同取值和连续变量两两组合成新特征后在不同card_id下的分组求和结果。接下来我们将其转化为DataFrame：

In [14]:
# 转化成df
df = pd.DataFrame(features).T.reset_index()

# 标注所有列
cols = df.columns.tolist()

# 修改df的特征名称
df.columns = ['card_id'] + cols[1:]
df

,card_id,A&1&C,A&1&D,B&2&C,B&2&D,A&2&C,A&2&D,B&1&C,B&1&D
0,1,5.0,11.0,5.0,11.0,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0
2,3,NaN,NaN,5.0,8.0,5.0,8.0,NaN,NaN


至此我们就完成了在极简数据集上进行通用组合特征的创建工作。

&emsp;&emsp;当然，通过上述过程不难发现，这种特征创建的方式能够非常高效的表示更多数据集中的隐藏信息，不过该方法容易产生较多空值，在后续建模过程中需要考虑特征矩阵过于稀疏从而带来的问题。

#### 1.2 基于transaction数据集创建通用组合特征

&emsp;&emsp;接下来，我们将上述过程应用于建模真实数据，即在此前已经清洗完的transaction数据集上来完成通用组合特征的创建工作。

- 数据读取

&emsp;&emsp;此处读取的transaction是此前创建的transaction_d_pre.csv数据集。

In [16]:
train = pd.read_csv('preprocess/train_pre.csv')
test =  pd.read_csv('preprocess/test_pre.csv')
transaction = pd.read_csv('preprocess/transaction_d_pre.csv')

- 字段类型标注

In [17]:
# 标注离散字段or连续型字段
numeric_cols = ['purchase_amount', 'installments']

category_cols = ['authorized_flag', 'city_id', 'category_1',
       'category_3', 'merchant_category_id','month_lag','most_recent_sales_range',
                 'most_recent_purchases_range', 'category_4',
                 'purchase_month', 'purchase_hour_section', 'purchase_day']

id_cols = ['card_id', 'merchant_id']

- 特征创建

In [21]:
# 创建字典用于保存数据
features = {}
card_all = train['card_id']._append(test['card_id']).values.tolist()
for card in card_all:
    features[card] = {}
     
# 标记不同类型字段的索引
columns = transaction.columns.tolist()
idx = columns.index('card_id')
category_cols_index = [columns.index(col) for col in category_cols]
numeric_cols_index = [columns.index(col) for col in numeric_cols]

# 记录运行时间
s = time.time()
num = 0

# 执行循环，并在此过程中记录时间
for i in range(transaction.shape[0]):
    va = transaction.loc[i].values
    card = va[idx]
    for cate_ind in category_cols_index:
        for num_ind in numeric_cols_index:
            col_name = '&'.join([columns[cate_ind], str(va[cate_ind]), columns[num_ind]])
            features[card][col_name] = features[card].get(col_name, 0) + va[num_ind]
    num += 1
    if num%1000000==0:
        print(time.time()-s, "s")
del transaction
gc.collect()

77.78152775764465 s
156.30035185813904 s
233.77025890350342 s
310.8533182144165 s
388.58491253852844 s
465.7131087779999 s
542.9651112556458 s
620.2659630775452 s
697.4840726852417 s
775.1001462936401 s
854.336339712143 s
931.5795841217041 s
1009.2299842834473 s
1086.400577545166 s
1163.4447948932648 s
1240.6105089187622 s
1317.9271388053894 s
1395.526971578598 s
1472.8169467449188 s
1550.1737365722656 s
1628.1112337112427 s
1705.3568155765533 s
1782.6971368789673 s
1860.14785695076 s
1937.7663321495056 s
2015.3726768493652 s
2093.225520133972 s
2171.1658523082733 s
2249.178324699402 s
2327.6118054389954 s
2406.600684404373 s


800

&emsp;&emsp;能够发现，整体运行所需时间较长。此外，此处需要注意的是，card_id的提取并不是从transaction从提取，而是从训练集和测试集中提取，大家想想看是什么原因？

&emsp;&emsp;在提取完特征后，接下来即可将带有交易数据特征的合并入训练集和测试集了：

In [22]:
# 字典转dataframe
df = pd.DataFrame(features).T.reset_index()
del features
cols = df.columns.tolist()
df.columns = ['card_id'] + cols[1:]

# 生成训练集与测试集
train = pd.merge(train, df, how='left', on='card_id')
test =  pd.merge(test, df, how='left', on='card_id')
del df
train.to_csv("preprocess/train_dict.csv", index=False)
test.to_csv("preprocess/test_dict.csv", index=False)

gc.collect()

0

&emsp;&emsp;至此，我们就完成了从transaction中提取通用特征的过程。简单查看数据集基本情况：

<center><img src="https://i.loli.net/2021/10/23/ZY75eSk3pAayoJn.png" alt="image-20211023161451438" style="zoom:67%;" />

### 2.业务统计特征创建

&emsp;&emsp;当然，除了通用组合特征外，我们还可以考虑从另一个角度进行特征提取，那就是先根据card_id来进行分组，然后统计不同字段再各组内的相关统计量，再将其作为特征，带入进行建模。其基本构造特征思路如下：

<center><img src="https://i.loli.net/2021/10/23/NupDc9JnBbHRPgU.png" alt="image-20211023162730619" style="zoom:80%;" />

该过程并不复杂，可以通过pandas中的groupby过程迅速实现。和此前特征构造的思路不同，通过该方法构造的特征，不会存在大量的缺失值，并且新增的列也将相对较少。代码实现过程如下：

- 数据读取：

In [23]:
transaction = pd.read_csv('preprocess/transaction_g_pre.csv')

- 字段类型标注

In [24]:
# 标注离散字段or连续型字段
numeric_cols = ['authorized_flag',  'category_1', 'installments',
       'category_3',  'month_lag','purchase_month','purchase_day','purchase_day_diff', 'purchase_month_diff',
       'purchase_amount', 'category_2', 
       'purchase_month', 'purchase_hour_section', 'purchase_day',
       'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']
categorical_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 'subsector_id']

- 特征提取过程

In [26]:
# 创建空字典
aggs = {}

# 连续/离散字段统计量提取范围
for col in numeric_cols:
    aggs[col] = ['nunique', 'mean', 'min', 'max','var','skew', 'sum']
for col in categorical_cols:
    aggs[col] = ['nunique']    
aggs['card_id'] = ['size', 'count']
cols = ['card_id']

# 借助groupby实现统计量计算
for key in aggs.keys():
# 将每个字段和对应的统计量拼接成新的特征列名，存储在 cols 中。
    cols.extend([key+'_'+stat for stat in aggs[key]])

df = transaction[transaction['month_lag']<0].groupby('card_id').agg(aggs).reset_index()
df.columns = cols[:1] + [co+'_hist' for co in cols[1:]]

df2 = transaction[transaction['month_lag']>=0].groupby('card_id').agg(aggs).reset_index()
df2.columns = cols[:1] + [co+'_new' for co in cols[1:]]
df = pd.merge(df, df2, how='left',on='card_id')

df2 = transaction.groupby('card_id').agg(aggs).reset_index()
df2.columns = cols
df = pd.merge(df, df2, how='left',on='card_id')
del transaction
gc.collect()

# 生成训练集与测试集
train = pd.merge(train, df, how='left', on='card_id')
test =  pd.merge(test, df, how='left', on='card_id')
del df
train.to_csv("preprocess/train_groupby.csv", index=False)
test.to_csv("preprocess/test_groupby.csv", index=False)

gc.collect()

0

执行完毕后，我们也可以简单查看数据集基本情况：

<center><img src="https://i.loli.net/2021/10/23/HpI1QuM6ZvtkS7f.png" alt="image-20211023162707542" style="zoom:67%;" />

### 3.数据合并

&emsp;&emsp;至此，我们即完成了从两个不同角度提取特征的相关工作。不过截至目前上述两套方案的特征仍然保存在不同数据文件中，我们需要对其进行合并，才能进一步带入进行建模，合并过程较为简单，只需要将train_dict(test_dict)与train_group(test_group)根据card_id进行横向拼接、然后剔除重复列即可，实现过程如下所示：

- 数据读取

In [27]:
train_dict = pd.read_csv("preprocess/train_dict.csv")
test_dict = pd.read_csv("preprocess/test_dict.csv")
train_groupby = pd.read_csv("preprocess/train_groupby.csv")
test_groupby = pd.read_csv("preprocess/test_groupby.csv")

- 剔除重复列

In [28]:
for co in train_dict.columns:
    if co in train_groupby.columns and co!='card_id':
        del train_groupby[co]
for co in test_dict.columns:
    if co in test_groupby.columns and co!='card_id':
        del test_groupby[co]

- 拼接特征

In [29]:
train = pd.merge(train_dict, train_groupby, how='left', on='card_id').fillna(0)
test = pd.merge(test_dict, test_groupby, how='left', on='card_id').fillna(0)

> 注，上述操作对缺失值进行了0的填补，此处缺失值并非真正的缺失值，该缺失值只是在特征创建过程没有统计结果的值，这些值从逻辑上来讲其实也都是0。因此此处缺失值填补相当于是数据补全。

- 数据保存与内存管理

In [30]:
train.to_csv("preprocess/train.csv", index=False)
test.to_csv("preprocess/test.csv", index=False)

del train_dict, test_dict, train_groupby, test_groupby
gc.collect()

0